# Preparación Dataset

In [ ]:
import os
import pandas as pd
from difflib import get_close_matches
import pprint
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score, classification_report
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import (
    accuracy_score, balanced_accuracy_score, confusion_matrix,
    precision_score, recall_score, f1_score, roc_auc_score, log_loss,
    classification_report
)
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import SMOTE



from sklearn.model_selection import RandomizedSearchCV, cross_val_predict
from sklearn.metrics import *
import optuna
from optuna.samplers import TPESampler
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.model_selection import StratifiedKFold

import matplotlib.pyplot as plt

import lime
import lime.lime_tabular
import shap



### Renombrar torneos
Inserta el directorio en el cual se encuentran todos los datasets y los renombra como "nombre_torneo_año"


In [ ]:
# Ruta del directorio donde están los archivos
#ruta donde se encuentran los csvs: 
directorio = './Toronto_M1000'
#Nombre con el que se renombraran los csv formato nombre_torneo_año
nombre_torneo = 'Toronto' 

In [ ]:
# Listar los archivos en el directorio
archivos = os.listdir(directorio)

# Renombrar los archivos
for i, archivo in enumerate(archivos):
    # Verificar que el archivo sea un archivo CSV
    if archivo.endswith('.csv'):
        print(f"Procesando archivo: {archivo}") 
        try:
            # Leer el archivo CSV con pandas
            df = pd.read_csv(os.path.join(directorio, archivo))
            
            # Verificar que la columna 'Date' exista en el archivo
            if 'Date' in df.columns:
                print(f"Columna 'Date' encontrada en {archivo}")  
                
                # Convertir la columna 'Date' a tipo datetime
                # Este paso maneja tanto fechas con 2 como 4 dígitos en el año (DD/MM/YY y DD/MM/YYYY)
                df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
                
                # Verificar si la conversión fue exitosa
                if df['Date'].isnull().any():
                    print(f"Advertencia: Algunas fechas no se pudieron convertir en {archivo}")  
                
                # Extraer el año del primer valor en la columna 'Date' (si es posible)
                anio = int(df['Date'].dt.year.iloc[0])  # Convertimos el año a entero
                
                print(f"Año extraído para {archivo}: {anio}")  

                # Obtenemos la extensión del archivo
                nombre, extension = os.path.splitext(archivo)
                
                # Generamos el nuevo nombre con el año extraído
                nuevo_nombre = f"{nombre_torneo}_{anio}{extension}"
                
                # Verificar si el nuevo nombre es distinto al original
                if nuevo_nombre != archivo:
                    # Renombramos el archivo
                    os.rename(os.path.join(directorio, archivo), os.path.join(directorio, nuevo_nombre))
                    print(f"Renombrado {archivo} a {nuevo_nombre}")  
                else:
                    print(f"El archivo {archivo} ya tiene el nombre correcto.") 
            else:
                print(f"No se encontró la columna 'Date' en {archivo}")  
        except Exception as e:
            print(f"Error al procesar el archivo {archivo}: {e}")  

print("Archivos procesados correctamente.")

### Unir todos los datasets en uno
Genera el dataset completo con la unión de todos los toreneos de todos los años con el nombre "nombre_torneo_completo"

In [ ]:

# Obtener la lista de archivos CSV en el directorio
archivos = [archivo for archivo in os.listdir(directorio) if archivo.endswith('.csv')]

# Crear una lista para almacenar los DataFrames de cada archivo CSV
dfs = []

# Leer cada archivo CSV y agregarlo a la lista
for archivo in archivos:
    df = pd.read_csv(os.path.join(directorio, archivo))
    dfs.append(df)

# Concatenar todos los DataFrames en uno solo
df_completo = pd.concat(dfs, ignore_index=True)

# Crear el nombre del archivo de salida
nombre_archivo_salida = f"{nombre_torneo}_completo.csv"

# Guardar el DataFrame combinado en el nuevo archivo CSV
df_completo.to_csv(os.path.join(ruta_salida, nombre_archivo_salida), index=False, sep=';', decimal=',')

print(f"Todos los archivos CSV han sido combinados exitosamente en '{nombre_archivo_salida}'.")


### Normalizar fechas formato YYYY/MM/DD

In [ ]:
# Ruta del archivo CSV combinado
ruta_fichero = './Toronto_M1000'
nombre_archivo_entrada = 'Toronto_completo.csv'
# Ruta del archivo CSV de salida
nombre_archivo_salida= 'Toronto_completo_normalizado.csv'
ruta_salida = './Masters1000'

In [ ]:
# Leer el archivo CSV en un DataFrame
df_completo = pd.read_csv(os.path.join(ruta_fichero, nombre_archivo_entrada),sep=';' )

# Función para convertir fechas con múltiples formatos
def convertir_fecha(fecha):
    formatos = ['%d/%m/%Y', '%d/%m/%y', '%Y-%m-%d']  # Formatos posibles
    for formato in formatos:
        try:
            return pd.to_datetime(fecha, format=formato)
        except ValueError:
            continue
    return pd.NaT  # Si no se puede convertir, devuelve NaT

# Aplicamos la función a toda la columna 'Date'
df_completo['Date'] = df_completo['Date'].astype(str).apply(convertir_fecha)

# Eliminamos filas donde la fecha sigue siendo NaT
df_completo = df_completo.dropna(subset=['Date'])

# Convertimos todas las fechas al formato `DD/MM/YYYY` 
df_completo['Date'] = df_completo['Date'].dt.strftime('%d/%m/%Y')

# Guardamos el archivo corregido
df_completo.to_csv(os.path.join(ruta_salida, nombre_archivo_salida), index=False, sep=';', decimal=',')

print(f"Archivo CSV con fechas normalizadas guardado en: {os.path.join(ruta_salida, nombre_archivo_salida)}")

### Unir todos los Torneos
Hay que tener en cuenta que los archivos por defecto tiene ',' pero una vez que lo hemos transformado los hemos guardado con ';' por lo que debemos leerlo de esta forma si no nos dara error y no leera correctamete las columnas

In [ ]:
# Ruta de entrada: carpeta donde están los CSV
directorio = './Modelo_Completo'  
# Ruta de salida: carpeta donde se guardará el CSV final
ruta_salida = './Modelo_Completo'  

In [ ]:
# Obtener todos los archivos .csv del directorio
archivos = [archivo for archivo in os.listdir(directorio) if archivo.endswith('.csv')]

# Lista para almacenar los DataFrames
dfs = []

# Leer y combinar todos los archivos
for archivo in archivos:
    ruta_completa = os.path.join(directorio, archivo)
    print(f"Leyendo: {archivo}")
    df = pd.read_csv(ruta_completa, sep=';')  # Usa ; como separador
    dfs.append(df)

# Unir todos los DataFrames
df_completo = pd.concat(dfs, ignore_index=True)

# Nombre del archivo combinado
nombre_archivo_salida = "Masters1000_completos_normalizados.csv"
ruta_final = os.path.join(ruta_salida, nombre_archivo_salida)

# Guardar el DataFrame combinado
df_completo.to_csv(ruta_final, index=False, sep=';', decimal=',')

print(f" Archivos combinados exitosamente en: {ruta_final}")


### Nombres inconsistentes
Al unir varios datasets el nombre de los jugadores puede que no coincidan y sean parecidos pero no iguales con este paso resolveremos esos casos

In [ ]:
# Obtener lista de nombres únicos de ganadores y perdedores
nombres_winners = df_completo['Winner'].dropna().unique()
nombres_losers = df_completo['Loser'].dropna().unique()

# Unificar todas las listas de nombres y convertirlos a minúsculas para mejor comparación
todos_nombres = {nombre.strip().lower() for nombre in nombres_winners}.union(
                {nombre.strip().lower() for nombre in nombres_losers})

# Diccionario para almacenar posibles inconsistencias
posibles_errores = {}

for nombre in todos_nombres:
    similares = get_close_matches(nombre, todos_nombres, n=5, cutoff=0.85)  # Ajustar el nivel de similitud
    similares = [s for s in similares if s != nombre]  # Eliminar coincidencias consigo mismo
    if similares:
        posibles_errores[nombre] = similares

# Mostrar posibles errores con mejor formato
if posibles_errores:
    print("\nPosibles nombres inconsistentes encontrados:\n")
    pprint.pprint(posibles_errores, width=100)
else:
    print("\nNo se encontraron inconsistencias en los nombres.")


### Corrección de nombres: Paso Manual


In [ ]:
# PASO MANUAL: Crear diccionario de correcciones según lo que elijas
#nombre incorrecto : nombre correcto
correcciones_manual = {
    "bogomolov jr. a.": "bogomolov jr.a.",
    "carreno busta p.": "carreno-busta p.",
    "del potro j. m.": "del potro j.m.",
    "dutra da silva r.": "dutra silva r.",
    "galan d.": "galan d.e.",
    "guzman j.": "guzman j.p.",
    "herbert p.h": "herbert p.h.",
    "kim k": "kim k.",
    "lisnard j.r..": "lisnard j.r.",
    "mathieu p.": "mathieu p.h.",
    "munoz de la nava d.": "munoz-de la nava d.",
    "o connell c.": "o'connell c.",
    "tseng c. h.": "tseng c.h.",
    "varillas j. p.": "varillas j.p.",
    "lisnard j.":"lisnard j.r.",
    "zhang ze":"zhang ze.",
    "chela j.i":"chela j.",
    "chela j.i.":"chela j.",
    "ferrero j.c.":"ferrero j.",
    "kuznetsov al.":"kuznetsov a.",
    "kuznetsov an.":"kuznetsov a.",
    "youzhny a.":"youzhny m." ,
    "bogomolov jr.a.":"bogomolov a." 
    
}

# Aplicar correcciones en el DataFrame
df_completo['Winner'] = df_completo['Winner'].str.strip().str.lower().replace(correcciones_manual)
df_completo['Loser'] = df_completo['Loser'].str.strip().str.lower().replace(correcciones_manual)

print("\nNombres corregidos manualmente aplicados en el DataFrame.")

Nombres corregidos manualmente aplicados en el DataFrame.


In [ ]:
# Definir la ruta base y el nombre del archivo de salida
ruta_base = r'./Modelo_Completo/nombres_corregidos'
nombre_archivo = 'GrandSlams_Masters1000_NombresCorregidos.csv'

In [ ]:

# Construir la ruta completa
ruta_salida = os.path.join(ruta_base, nombre_archivo)
# Guardar el DataFrame en un archivo CSV
df_completo.to_csv(ruta_salida,  index=False, sep=';', decimal=',')

### Cargar dataframe para añadir columnas

In [ ]:
# Ruta base del archivo
ruta_base = r"./Modelo_Completo/columnas_añadidas"
# Nombre del archivo
nombre_archivo = "escaladofinal.csv"

In [ ]:
ruta_completa = os.path.join(ruta_base, nombre_archivo)

# Leer el CSV correctamente
df = pd.read_csv(
    ruta_completa,
    delimiter=";",   # separador de columnas
    decimal=","      # separador decimal 
)

# Convertir fecha
df["Date"] = pd.to_datetime(df["Date"], format="%d/%m/%Y", errors="coerce")

# Ordenar por fecha
df = df.sort_values(by="Date")

# Mostrar resumen
df.info()


## Correccion Wrank

In [ ]:
for col in ["WRank", "LRank"]:
    df[col] = (
        df[col]
          .astype(str)           # asegurar que es texto
          .str.strip()           # quitar espacios
          .str.split(",").str[0] # tomar lo que va antes de la coma
          .replace({"UNR": None, "NR": None, "": None})
          .pipe(pd.to_numeric, errors="coerce")
    )


## Añadir Varibles nuevas

## Jugador favorito y no favorito
El objetivo es crear dos columnas una con el jugador favorito(jugador que ocupa mejor posición en el ranking) y otra con el jugador que no es favorito(ocupa peor posición en el ranking).

In [ ]:
# Asegurar que los rankings sean numéricos
df["WRank"] = pd.to_numeric(df["WRank"], errors="coerce")
df["LRank"] = pd.to_numeric(df["LRank"], errors="coerce")

# Inicializar listas para jugadores
favorito = []
no_favorito = []

# Crear columnas según el ranking
for _, row in df.iterrows():
    w = row["Winner"]
    l = row["Loser"]
    wr = row["WRank"]
    lr = row["LRank"]

    if pd.isna(wr) and pd.isna(lr):
        favorito.append(None)
        no_favorito.append(None)
    elif pd.isna(wr):
        favorito.append(l)
        no_favorito.append(w)
    elif pd.isna(lr):
        favorito.append(w)
        no_favorito.append(l)
    elif wr < lr:
        favorito.append(w)
        no_favorito.append(l)
    else:
        favorito.append(l)
        no_favorito.append(w)

# Insertar justo después de 'Best of'
idx = df.columns.get_loc("Best of")
df.insert(idx + 1, "Player_Favorite", favorito)
df.insert(idx + 2, "Player_Not_Favorite", no_favorito)

In [ ]:
ruta_base = r"./Modelo_Completo/columnas_añadidas" 
nombre_archivo = 'jugadoresAñadidos.csv'

In [ ]:
# Construir la ruta completa
ruta_salida = os.path.join(ruta_base, nombre_archivo)

# Guardar el DataFrame en un archivo CSV
df.to_csv(ruta_salida, index=False, sep=';', decimal=',')

## Varibale Favorite_Wins
Indica si el jugador favorito ha ganado (1) o si ha perdido (0)

In [ ]:
# Inicializar lista para la variable objetivo
favorite_wins = []

# Usar las columnas ya creadas
for _, row in df.iterrows():
    if pd.isna(row["Player_Favorite"]) or pd.isna(row["Winner"]):
        favorite_wins.append(None)
    elif row["Player_Favorite"] == row["Winner"]:
        favorite_wins.append(1)
    else:
        favorite_wins.append(0)

# Insertar la variable justo después de 'Player_Not_Favorite'
idx = df.columns.get_loc("Player_Not_Favorite")
df.insert(idx + 1, "Favorite_Wins", favorite_wins)


In [ ]:
ruta_base = r"./Modelo_Completo/columnas_añadidas" 
nombre_archivo = 'favorite_wins.csv'

In [ ]:
# Construir la ruta completa
ruta_salida = os.path.join(ruta_base, nombre_archivo)

# Guardar el DataFrame en un archivo CSV
df.to_csv(ruta_salida, index=False, sep=';', decimal=',')

## Rellenar columnas vacias WRank/LRank
Para asegurarnos de que todos los jugadores tienen ranking rellenamos los jugadores que no lo tienen con un ranking muy alto

In [ ]:
# Rellenar los valores nulos en 'WRank' y 'LRank' con 5000
df['WRank'] = df['WRank'].fillna(5000)
df['LRank'] = df['LRank'].fillna(5000)

# Verificar que los valores nulos han sido rellenados
print(df[['WRank', 'LRank']].isnull().sum())

In [ ]:
ruta_base = r"./Modelo_Completo/columnas_añadidas" 
nombre_archivo = 'columnas_vacias.csv'

In [ ]:
# Construir la ruta completa
ruta_salida = os.path.join(ruta_base, nombre_archivo)

# Guardar el DataFrame en un archivo CSV
df.to_csv(ruta_salida, index=False, sep=';', decimal=',')

## Rank_Favorite/Rank_Not_Favorite/Rank_Diff_Signed/Rank_Diff_Abs
Rank_favorite = Representa el ranking del jugador favorito  
Rank_Not_Favorite = Representa el ranking del jugador que no es favorito  
Rank_Diff_Signed = Representa la diferencia con signo entre el jugador favorito y el no favorito  
Rank_Diff_Abs = Representa la diferencia entre el jugador favorito y el no favorito en valor absoluto  

In [ ]:
# Asegurar tipos numéricos
df["WRank"] = pd.to_numeric(df["WRank"], errors="coerce")
df["LRank"] = pd.to_numeric(df["LRank"], errors="coerce")

# Inicializar listas
rank_fav = []
rank_not_fav = []

# Calcular rankings en base al favorito
for _, row in df.iterrows():
    fav = row["Player_Favorite"]
    win = row["Winner"]
    wr = row["WRank"]
    lr = row["LRank"]

    if pd.isna(fav):
        rank_fav.append(None)
        rank_not_fav.append(None)
    elif fav == win:
        rank_fav.append(wr)
        rank_not_fav.append(lr)
    else:
        rank_fav.append(lr)
        rank_not_fav.append(wr)

# Crear columnas temporales
df["Rank_Favorite"] = rank_fav
df["Rank_Not_Favorite"] = rank_not_fav
df["Rank_Diff_Signed"] = df["Rank_Favorite"] - df["Rank_Not_Favorite"]
df["Rank_Diff_Abs"] = df["Rank_Diff_Signed"].abs()

# Reordenar: insertar después de Player_Favorite y Player_Not_Favorite
idx = df.columns.get_loc("Player_Not_Favorite")

for col in ["Rank_Favorite", "Rank_Not_Favorite", "Rank_Diff_Signed", "Rank_Diff_Abs"]:
    temp = df.pop(col)
    idx += 1
    df.insert(idx, col, temp)



In [ ]:
ruta_base = r"./Modelo_Completo/columnas_añadidas" 
nombre_archivo = 'ranking_favorito.csv'

In [ ]:
# Construir la ruta completa
ruta_salida = os.path.join(ruta_base, nombre_archivo)

# Guardar el DataFrame en un archivo CSV
df.to_csv(ruta_salida, index=False, sep=';', decimal=',')

## WinStreak_Favorite / WinStreak_Not_Favorite
WinStreak_Favorite: Representa cuántos partidos consecutivos ha ganado el jugador favorito antes del partido actual.  
WinStreak_Not_Favorite: Representa cuántos partidos consecutivos ha ganado el jugador no favorito antes del partido actual.

Recuerda cargar el fichero mas reciente

In [ ]:
# Asegurarse de que la columna 'Date' está en formato datetime y ordenada
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date')

# Diccionario para guardar racha de cada jugador
win_streaks = {}

# Nuevas listas para almacenar la racha antes del partido
streak_fav = []
streak_not_fav = []

# Recorrer el DataFrame
for _, row in df.iterrows():
    fav = row['Player_Favorite']
    no_fav = row['Player_Not_Favorite']
    winner = row['Winner']

    # Obtener las rachas antes del partido
    streak_fav_val = win_streaks.get(fav, 0)
    streak_not_fav_val = win_streaks.get(no_fav, 0)

    streak_fav.append(streak_fav_val)
    streak_not_fav.append(streak_not_fav_val)

    # Actualizar rachas después del partido
    if fav == winner:
        win_streaks[fav] = streak_fav_val + 1
        win_streaks[no_fav] = 0
    else:
        win_streaks[no_fav] = streak_not_fav_val + 1
        win_streaks[fav] = 0

# Añadir columnas al DataFrame
df['WinStreak_Favorite'] = streak_fav
df['WinStreak_Not_Favorite'] = streak_not_fav

# Insertar justo después de Player_Not_Favorite
idx = df.columns.get_loc("Player_Not_Favorite")
df.insert(idx + 1, "WinStreak_Favorite", df.pop("WinStreak_Favorite"))
df.insert(idx + 2, "WinStreak_Not_Favorite", df.pop("WinStreak_Not_Favorite"))


In [ ]:
# Definir la ruta base y el nombre del archivo de salida
ruta_base = r"./Modelo_Completo/columnas_añadidas" 
nombre_archivo = 'winStreak.csv'

In [ ]:

# Construir la ruta completa
ruta_salida = os.path.join(ruta_base, nombre_archivo)

# Guardar el DataFrame en un archivo CSV
df.to_csv(ruta_salida, index=False, sep=';', decimal=',')

## GrandSlams_Favorite y GrandSlams_Not_Favorite
Representa el número total de GrandSlams ganados por el jugador favorito y el no favorito antes del partido.

In [ ]:

# Asegurarse de que la columna 'Date' está en formato datetime y ordenar cronológicamente
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df = df.sort_values('Date')

# Inicializar diccionario para contar Grand Slams ganados por jugador
grand_slam_titles = {}

# Listas para almacenar resultados por fila
gs_fav = []
gs_nofav = []

# Iterar por filas del DataFrame
for _, row in df.iterrows():
    fav = row['Player_Favorite']
    nofav = row['Player_Not_Favorite']
    
    # Inicializar si no existe
    if fav not in grand_slam_titles:
        grand_slam_titles[fav] = 0
    if nofav not in grand_slam_titles:
        grand_slam_titles[nofav] = 0
    
    # Guardar títulos antes del partido
    gs_fav.append(grand_slam_titles[fav])
    gs_nofav.append(grand_slam_titles[nofav])
    
    # Si es una final de Grand Slam, añadir uno al ganador
    if row['Round'] == 'The Final' and row['Series'] == 'Grand Slam':
        if row['Favorite_Wins'] == 1:
            grand_slam_titles[fav] += 1
        else:
            grand_slam_titles[nofav] += 1

# Añadir las nuevas columnas al DataFrame
df['GrandSlams_Favorite'] = gs_fav
df['GrandSlams_Not_Favorite'] = gs_nofav

# Reordenar columnas para que queden justo después de Favorite_Wins
cols = list(df.columns)
favwins_index = cols.index('Favorite_Wins')

# Insertar las nuevas columnas en la posición deseada
cols.remove('GrandSlams_Favorite')
cols.remove('GrandSlams_Not_Favorite')
cols.insert(favwins_index + 1, 'GrandSlams_Favorite')
cols.insert(favwins_index + 2, 'GrandSlams_Not_Favorite')

# Reordenar el DataFrame
df = df[cols]

# Verificar ejemplo
print(df[['Player_Favorite', 'Player_Not_Favorite', 'Favorite_Wins', 'GrandSlams_Favorite', 'GrandSlams_Not_Favorite']].head())



In [ ]:
ruta_base = r"./Modelo_Completo/columnas_añadidas" 
nombre_archivo = 'GrandSlams.csv'

In [ ]:

# Construir la ruta completa
ruta_salida = os.path.join(ruta_base, nombre_archivo)

# Guardar el DataFrame en un archivo CSV
df.to_csv(ruta_salida, index=False, sep=';', decimal=',')

## Master1000_Favorite y Master1000_Not_Favorite
Representa el número total de master1000 ganados por el jugador favorito y el no favorito antes del partido.

In [ ]:
# 2. Normalizar fecha y ordenar
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
df = df.sort_values("Date")

# 3. Diccionario donde iremos acumulando títulos Masters 1000 por jugador
masters_titles = {}

# 4. Listas para ir guardando el nº de títulos que cada jugador llevaba **antes** de cada partido
m1000_fav  = []
m1000_nfav = []

for _, row in df.iterrows():
    fav   = row["Player_Favorite"]
    nfav  = row["Player_Not_Favorite"]
    
    # Inicializar si no existen todavía
    if fav not in masters_titles:
        masters_titles[fav] = 0
    if nfav not in masters_titles:
        masters_titles[nfav] = 0
    
    # Guardar el recuento antes de este partido
    m1000_fav.append(masters_titles[fav])
    m1000_nfav.append(masters_titles[nfav])
    
    # ¿Es la final de un Masters 1000?
    if row["Round"] == "The Final" and row["Series"] in ["Masters", "Masters 1000"]:
        if row["Favorite_Wins"] == 1:
            masters_titles[fav] += 1       # ganó el favorito
        else:
            masters_titles[nfav] += 1      # ganó el no-favorito

# 5. Colocar las nuevas columnas en el DataFrame
df["Masters1000_Favorite"]     = m1000_fav
df["Masters1000_Not_Favorite"] = m1000_nfav

# 6. (Opcional) reordenar las columnas para que queden justo después de Favorite_Wins
cols            = list(df.columns)
favwins_idx     = cols.index("Favorite_Wins")
cols.remove("Masters1000_Favorite")
cols.remove("Masters1000_Not_Favorite")
cols.insert(favwins_idx + 1, "Masters1000_Favorite")
cols.insert(favwins_idx + 2, "Masters1000_Not_Favorite")
df = df[cols]

# 7. Comprobación rápida
print(
    df[
        [
            "Player_Favorite",
            "Player_Not_Favorite",
            "Favorite_Wins",
            "Masters1000_Favorite",
            "Masters1000_Not_Favorite",
        ]
    ].head()
)

In [ ]:
ruta_base = r"./Modelo_Completo/columnas_añadidas" 
nombre_archivo = 'Master1000titles.csv'

In [ ]:

# Construir la ruta completa
ruta_salida = os.path.join(ruta_base, nombre_archivo)

# Guardar el DataFrame en un archivo CSV
df.to_csv(ruta_salida, index=False, sep=';', decimal=',')

## Surface_WinRate_Favorite / Surface_WinRate_Not_Favorite / Surface_Matches_Favorite / Surface_Matches_Not_Favorite /Favorite Wins
Representa el ratio de victorias de cada jugador en una superficie, es decir de los partidos que un jugador a jugado en una superficie, cuantos de ellos ha ganado  
Suface_Matches_Favorite/Not_Favorite: Representa el número de partidos que ha jugado cada jugador en esa superficie.
FavoriteWins indica si el jugador favorito ha ganado o no.


In [ ]:
# Diccionario para almacenar historial por superficie
surface_history = {}

# Listas para las columnas
surface_winrate_fav = []
surface_winrate_nofav = []
surface_matches_fav = []
surface_matches_nofav = []

# Iterar por el DataFrame
for _, row in df.iterrows():
    fav = row['Player_Favorite']
    nofav = row['Player_Not_Favorite']
    surface = row['Surface']
    
    for player in [fav, nofav]:
        if player not in surface_history:
            surface_history[player] = {}
        if surface not in surface_history[player]:
            surface_history[player][surface] = {'wins': 0, 'total': 0}
    
    fav_stats = surface_history[fav][surface]
    nofav_stats = surface_history[nofav][surface]

    # Winrate (si nunca jugó, poner 0.0)
    wr_fav = fav_stats['wins'] / fav_stats['total'] if fav_stats['total'] > 0 else 0.0
    wr_nofav = nofav_stats['wins'] / nofav_stats['total'] if nofav_stats['total'] > 0 else 0.0

    surface_winrate_fav.append(wr_fav)
    surface_winrate_nofav.append(wr_nofav)
    surface_matches_fav.append(fav_stats['total'])
    surface_matches_nofav.append(nofav_stats['total'])

    # Actualizar historial
    if row['Favorite_Wins'] == 1:
        surface_history[fav][surface]['wins'] += 1
    else:
        surface_history[nofav][surface]['wins'] += 1

    surface_history[fav][surface]['total'] += 1
    surface_history[nofav][surface]['total'] += 1

# Añadir columnas nuevas
df['Surface_WinRate_Favorite'] = surface_winrate_fav
df['Surface_WinRate_Not_Favorite'] = surface_winrate_nofav
df['Surface_Matches_Favorite'] = surface_matches_fav
df['Surface_Matches_Not_Favorite'] = surface_matches_nofav

# Insertar después de 'Surface'
cols = list(df.columns)
surface_idx = cols.index('Surface')
for col in ['Surface_WinRate_Favorite', 'Surface_WinRate_Not_Favorite', 'Surface_Matches_Favorite', 'Surface_Matches_Not_Favorite']:
    cols.remove(col)
cols[surface_idx + 1:surface_idx + 1] = ['Surface_WinRate_Favorite', 'Surface_WinRate_Not_Favorite', 'Surface_Matches_Favorite', 'Surface_Matches_Not_Favorite']
df = df[cols]

# Verificamos las nuevas columnas
print(df[['Player_Favorite', 'Player_Not_Favorite', 'Surface',
          'Surface_WinRate_Favorite', 'Surface_WinRate_Not_Favorite',
          'Surface_Matches_Favorite', 'Surface_Matches_Not_Favorite']].head())


In [ ]:
ruta_base = r"./Modelo_Completo/columnas_añadidas" 
nombre_archivo = 'WinRateSurface.csv'

In [ ]:
# Construir la ruta completa
ruta_salida = os.path.join(ruta_base, nombre_archivo)

# Guardar el DataFrame en un archivo CSV
df.to_csv(ruta_salida, index=False, sep=';', decimal=',')

## Round Code
Codifica las rondas en formato numérico

In [ ]:
# Mapeo de rondas de Grand Slam a valores numéricos
round_mapping = {
    '1st Round': 1,
    '2nd Round': 2,
    '3rd Round': 3,
    '4th Round': 4,
    'Quarterfinals': 5,
    'Semifinals': 6,
    'The Final': 7
}

# Crear nueva columna 'Round_Code'
df['Round_Code'] = df['Round'].map(round_mapping)

# Reordenar: insertar justo después de la columna 'Round'
cols = list(df.columns)
round_index = cols.index('Round')
cols.remove('Round_Code')
cols.insert(round_index + 1, 'Round_Code')
df = df[cols]


In [ ]:
ruta_base = r"./Modelo_Completo/columnas_añadidas" 
nombre_archivo = 'RoundCode.csv'

In [ ]:
# Construir la ruta completa
ruta_salida = os.path.join(ruta_base, nombre_archivo)

# Guardar el DataFrame en un archivo CSV
df.to_csv(ruta_salida, index=False, sep=';', decimal=',')

## Win_Streak_Diff
Diferencia de racha de victorias entre un jugador y otro

In [ ]:
# Asegurarse de que las columnas son numéricas
df["WinStreak_Favorite"] = pd.to_numeric(df["WinStreak_Favorite"], errors="coerce")
df["WinStreak_Not_Favorite"] = pd.to_numeric(df["WinStreak_Not_Favorite"], errors="coerce")

# Crear la columna como diferencia de rachas
df["Win_Streak_Diff"] = df["WinStreak_Favorite"] - df["WinStreak_Not_Favorite"]

# Reordenar para insertar justo después de WinStreak_Not_Favorite
cols = list(df.columns)
idx = cols.index("WinStreak_Not_Favorite")
cols.remove("Win_Streak_Diff")
cols.insert(idx + 1, "Win_Streak_Diff")
df = df[cols]

# Verifica las columnas relevantes
print(df[["Player_Favorite", "WinStreak_Favorite", "Player_Not_Favorite", "WinStreak_Not_Favorite", "Win_Streak_Diff"]].head())


In [ ]:
ruta_base = r"./Modelo_Completo/columnas_añadidas" 
nombre_archivo = 'diff_win_Streak.csv'

In [ ]:
# Construir la ruta completa
ruta_salida = os.path.join(ruta_base, nombre_archivo)

# Guardar el DataFrame en un archivo CSV
df.to_csv(ruta_salida, index=False, sep=';', decimal=',')

## ATP_Points_Favorite / ATP_Points_Not_Favorite
Representan los puntos ATP del jugador favorito y del no favorito

In [ ]:

# Asegurarse de que los nombres de columna estén bien 
# Convertir WPts y LPts a numéricos por seguridad
df["WPts"] = pd.to_numeric(df["WPts"], errors="coerce")
df["LPts"] = pd.to_numeric(df["LPts"], errors="coerce")

# Crear las columnas según quién es el favorito
fav_pts = []
nofav_pts = []

for _, row in df.iterrows():
    if row["Player_Favorite"] == row["Winner"]:
        fav_pts.append(row["WPts"])
        nofav_pts.append(row["LPts"])
    else:
        fav_pts.append(row["LPts"])
        nofav_pts.append(row["WPts"])

# Agregar al DataFrame
df["ATP_Points_Favorite"] = fav_pts
df["ATP_Points_Not_Favorite"] = nofav_pts

# Reordenar para colocarlas justo después de 'Comment'
cols = list(df.columns)
idx = cols.index("Comment")
cols.remove("ATP_Points_Favorite")
cols.remove("ATP_Points_Not_Favorite")
cols.insert(idx + 1, "ATP_Points_Favorite")
cols.insert(idx + 2, "ATP_Points_Not_Favorite")
df = df[cols]

# Verificar resultado
print(df[["Player_Favorite", "Player_Not_Favorite", "ATP_Points_Favorite", "ATP_Points_Not_Favorite"]].head())



In [ ]:
ruta_base = r"./Modelo_Completo/columnas_añadidas" 
nombre_archivo = 'Points_favorite.csv'

In [ ]:
# Construir la ruta completa
ruta_salida = os.path.join(ruta_base, nombre_archivo)

# Guardar el DataFrame en un archivo CSV
df.to_csv(ruta_salida, index=False, sep=';', decimal=',')

## Juegos Totales

In [ ]:
import pandas as pd

# Asegurarse de que estas columnas existen y están en el DataFrame
cols_ganador = ["W1", "W2", "W3", "W4", "W5"]
cols_perdedor = ["L1", "L2", "L3", "L4", "L5"]

# Convertir a numérico, por si hay strings o NaNs
for col in cols_ganador + cols_perdedor:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# Crear la nueva columna sumando todos los juegos jugados
df["Total_Games"] = df[cols_ganador + cols_perdedor].sum(axis=1)

# Reubicar la nueva columna justo después de 'Lsets'
cols = list(df.columns)
idx = cols.index("Lsets") + 1
cols.remove("Total_Games")
cols.insert(idx, "Total_Games")
df = df[cols]


In [ ]:
ruta_base = r"./Modelo_Completo/columnas_añadidas" 
nombre_archivo = 'JuegosTotales.csv'

In [ ]:
# Construir la ruta completa
ruta_salida = os.path.join(ruta_base, nombre_archivo)

# Guardar el DataFrame en un archivo CSV
df.to_csv(ruta_salida, index=False, sep=';', decimal=',')

## Codificacion variables categoricas Surface y Comment

In [ ]:
# --- Codificación de Surface ---
# Crear variables dummy para 'Surface' sin eliminar la columna original
df_surface_dummies = pd.get_dummies(df["Surface"], prefix="Surface")

# Obtener índice de la columna original 'Surface'
surface_index = df.columns.get_loc("Surface")

# Insertar columnas dummy justo después de 'Surface'
for i, col in enumerate(df_surface_dummies.columns):
    df.insert(surface_index + 1 + i, col, df_surface_dummies[col])

# --- Codificación de Comment ---
# Crear columna binaria 'Completed' según si el partido fue 'Completed'
df["Completed"] = (df["Comment"].str.lower() == "completed").astype(int)

# Reubicar 'Completed' justo después de 'Comment'
comment_index = df.columns.get_loc("Comment")
col_completed = df.pop("Completed")  # eliminar temporalmente
df.insert(comment_index + 1, "Completed", col_completed)  # volver a insertar en la posición correcta


In [ ]:
ruta_base = r"./Modelo_Completo/columnas_añadidas" 
nombre_archivo = 'surface_comment.csv'

In [ ]:
# Construir la ruta completa
ruta_salida = os.path.join(ruta_base, nombre_archivo)

# Guardar el DataFrame en un archivo CSV
df.to_csv(ruta_salida, index=False, sep=';', decimal=',')

## Pierde el favorito

In [ ]:
# Asegúrate de que la columna 'Favorite_Wins' existe
if "Favorite_Wins" in df.columns:
    # Si la columna ya existe, elimínala para evitar errores al insertar
    if "Favorite_Loses" in df.columns:
        df.drop(columns="Favorite_Loses", inplace=True)
    
    # Crear nueva columna: 1 si pierde el favorito, 0 si gana
    loses_col = (1 - df["Favorite_Wins"]).astype(int)
    df.insert(
        loc=df.columns.get_loc("Favorite_Wins") + 1,
        column="Favorite_Loses",
        value=loses_col
    )
else:
    print("No se encontró la columna 'Favorite_Wins'. Verifica el nombre.")




In [ ]:
ruta_base = r"./Modelo_Completo/columnas_añadidas" 
nombre_archivo = 'datasetFinal.csv'

In [ ]:
# Construir la ruta completa
ruta_salida = os.path.join(ruta_base, nombre_archivo)

# Guardar el DataFrame en un archivo CSV
df.to_csv(ruta_salida, index=False, sep=';', decimal=',')

## Escalado de variables
Se usará StandardScaler para el escalado de variables

In [ ]:
# Lista de columnas a escalar
features_to_scale = [
    "Surface_WinRate_Favorite", "Surface_WinRate_Not_Favorite",
    "Surface_Matches_Favorite", "Surface_Matches_Not_Favorite",
    "WinStreak_Favorite", "WinStreak_Not_Favorite", "Win_Streak_Diff",
    "Rank_Favorite", "Rank_Not_Favorite", "Rank_Diff_Signed", "Rank_Diff_Abs",
    "GrandSlams_Favorite", "GrandSlams_Not_Favorite",
    "Masters1000_Favorite","Masters1000_Not_Favorite"
]

# Asegurarse de que son numéricos
for col in features_to_scale:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# Escalado con StandardScaler
scaler = StandardScaler()
df[features_to_scale] = scaler.fit_transform(df[features_to_scale])


In [ ]:
ruta_base = r"./Modelo_Completo/columnas_añadidas" 
nombre_archivo = 'escaladofinal.csv'

In [ ]:
# Construir la ruta completa
ruta_salida = os.path.join(ruta_base, nombre_archivo)

# Guardar el DataFrame en un archivo CSV
df.to_csv(ruta_salida, index=False, sep=';', decimal=',')